In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
from collections import Counter
from sklearn.metrics import accuracy_score, classification_report
import pickle
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import StandardScaler

In [3]:
data = pd.read_csv("train_test_datasetV2.csv")

In [4]:
print(len(data))
print(data['target'].value_counts())
print(data.columns)
documents = data['clean_tweet'].to_list()
labels = data['target'].to_list()

print(f'Type of documents {type(documents)} and labels: {type(labels)}')
print(f'Length {len(documents)} and labels: {len(labels)}')

6008
target
0    3715
1    2293
Name: count, dtype: int64
Index(['ID', 'tweet', 'clean_tweet', 'target'], dtype='object')
Type of documents <class 'list'> and labels: <class 'list'>
Length 6008 and labels: 6008


In [5]:
# Step 1: Select Seed Documents
seed_size = 1000
seed_indices = np.random.choice(range(len(documents)), size=seed_size, replace=False)
seed_documents = [documents[i] for i in seed_indices]
seed_labels = [labels[i] for i in seed_indices]

print(len(seed_documents))
distribution = Counter(seed_labels)
print("Value count of Seed Documents")
print(distribution)

1000
Value count of Seed Documents
Counter({0: 595, 1: 405})


In [6]:
class_0_indices = [i for i, label in enumerate(labels) if label == 0]
class_1_indices = [i for i, label in enumerate(labels) if label == 1]

seed_size = 1000

# Define seed size for each class
seed_size_per_class = seed_size // 2

# Sample equally from each class
seed_indices_0 = np.random.choice(class_0_indices, size=seed_size_per_class, replace=False)
seed_indices_1 = np.random.choice(class_1_indices, size=seed_size_per_class, replace=False)

# Combine the samples
seed_indices = np.concatenate((seed_indices_0, seed_indices_1))
np.random.shuffle(seed_indices)  # Shuffle to mix the classes

# Extract seed documents and labels
seed_documents = [documents[i] for i in seed_indices]
seed_labels = [labels[i] for i in seed_indices]

# Checking the distribution
print("Number of Seed Documents:", len(seed_documents))
distribution = Counter(seed_labels)
print("Value count of Seed Documents:")
print(distribution)

Number of Seed Documents: 1000
Value count of Seed Documents:
Counter({0: 500, 1: 500})


In [7]:
# Define TFIDF Vectorizer

vectorizer = TfidfVectorizer()
X_seed = vectorizer.fit_transform(seed_documents)

scaler = StandardScaler(with_mean=False)
X_seed_scaled = scaler.fit_transform(X_seed)

# Initialize Logistic Regression 
model = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

# Fit the model
model.fit(X_seed_scaled, seed_labels)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(max_iter=1000, solver='saga')

In [8]:
total_budget = 5000
batch_size = 150
remaining_budget = total_budget - len(seed_documents)

# Active Learning Loop using SAL
while remaining_budget > 0:
    X = vectorizer.transform(documents)
    X_scaled = scaler.transform(X)
    probabilities = model.predict_proba(X_scaled)[:, 1]  # Logistic Regression on CPU

    # SAL: Select documents with probabilities closest to 0.5
    uncertain_indices = np.argsort(np.abs(probabilities - 0.5))[:batch_size]

    # Simulate annotation
    annotated_labels = [labels[i] for i in uncertain_indices]

    # Retrain the model with selected documents
    X_train = vectorizer.transform([documents[i] for i in selected_indices])
    X_train = ectorizer.transform([documents[i] for i in uncertain_indices])
    X_train_scaled = scaler.transform(X_train)
    model.fit(X_train_scaled, annotated_labels)
    
    remaining_budget -= batch_size
    print(f'Remaining budget: {remaining_budget}')

NameError: name 'selected_indices' is not defined

In [ ]:
# Start the testing
exp_data = pd.read_csv("experiment_datasetV2.csv")

print(len(exp_data))
print(exp_data['target'].value_counts())
print(exp_data.columns)

test_documents = exp_data['clean_tweet'].to_list()
test_labels = exp_data['target'].to_list()
test_IDs = exp_data['ID'].to_list()

print(f'Type of documents {type(test_documents)} and labels: {type(test_labels)}')
print(f'Length {len(test_documents)} and labels: {len(test_labels)}')

In [ ]:
X_test = vectorizer.transform(test_documents)
X_test_scaled = scaler.transform(X_test)

# Predict using the trained model
predicted_labels = model.predict(X_test_scaled) # <--- change here
predicted_probabilities = model.predict_proba(X_test_scaled) # <--- change here
hateful_probabilities = predicted_probabilities[:, 1]

distribution = Counter(predicted_labels)
print("Value count of predicted_labels")
print(distribution)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(test_labels, predicted_labels)
report = classification_report(test_labels, predicted_labels)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

In [ ]:
tfidf_SAL_experiment = pd.DataFrame({
    'ID': test_IDs,
    'clean_tweet': test_documents,
    'true_label': test_labels,
    'predicted_label': predicted_labels,
    'probability_of_hateful': predicted_probabilities[:, 1]
})

In [ ]:
tfidf_SAL_experiment.to_csv("tfidf_SAL_experiment_results.csv", index=False)

with open('logistic_regression_model_tfidf_SAL.pkl', 'wb') as file:
    pickle.dump(model, file)  #<--- change here

print("Done")